In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ADD https://drive.google.com/file/d/1L1PSvBV3-hfyOCwPyjZBzBVTMhd7MF8a/view

In [3]:
!unzip /content/drive/MyDrive/in.zip -d /content

Archive:  /content/drive/MyDrive/in.zip
  inflating: /content/in.csv         


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=b8fcdf60c496c4dcf7a18f0cfb3ce66ce4c35bd2dcf47e6498219dfa8e67e79c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
!pip install findspark

In [6]:
import findspark
findspark.init() #check spark installation

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [7]:
# Create Spark Session
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("CardReport")\
    .getOrCreate()

In [8]:
# Create Spark Context
sc=spark.sparkContext

In [9]:
# # Read Input into RDD
rdd=sc.textFile('/content/in.csv')
# # Show first 10 rows
print(rdd.take(3))
print(rdd.count())

['224,10', '836,100', '81,5']
75000000


In [11]:
# Split row to be tuple of 2 elements the id and the value of the card
rdd_split=rdd.map(lambda x: tuple(x.split(',')))
print(rdd_split.take(3))

[('224', '10'), ('836', '100'), ('81', '5')]


In [12]:
# Reduce the rdd by key :D first element of the tuple
rdd_result=rdd_split.reduceByKey(lambda x,y:int(x)+int(y)).sortByKey()
print(rdd_result.take(3))

[('0', 2554710), ('1', 2540660), ('10', 2525035)]


In [13]:
# Convert Tuple to string space separted :D
rdd_csv = rdd_result.map(lambda x: ' '.join(map(str, x)))
rdd_csv.take(10)

['0 2554710',
 '1 2540660',
 '10 2525035',
 '100 2568385',
 '101 2564425',
 '102 2538315',
 '103 2564125',
 '104 2555395',
 '105 2569490',
 '106 2555705']

In [14]:
# Save Result in 1 Partition
rdd_csv.coalesce(1).saveAsTextFile("out")

In [16]:
# Rename output file
import os
os.rename("out/part-00000", "output.csv")